# HBR transfer example

Welcome to this example/tutorial notebook that will go through the fitting, evaluation, and transfering of HBR models. 

### Imports

In [1]:
import os
import pickle
import pcntoolkit as ptk
import pandas as pd
import numpy as np
import pprint
import arviz as az
import matplotlib.pyplot as plt
import pymc as pm
import pytensor as pt
from pytensor.printing import pydotprint
from pcntoolkit.dataio.norm_data import NormData
from pcntoolkit.normative_model.norm_conf import NormConf
from pcntoolkit.normative_model.norm_hbr import NormHBR
from pcntoolkit.normative_model.norm_factory import load_normative_model
from pcntoolkit.normative_model.norm_factory import create_normative_model
from pcntoolkit.regression_model.hbr.hbr_conf import HBRConf
from pcntoolkit.regression_model.hbr.param import Param
import seaborn as sns
from pytensor.compile.profiling import ProfileStats

# Load data

In [2]:
savedir = "/home/guus/Desktop/pcn_profile_data"

def load_data():
    with open(os.path.join(savedir, 'train_data'), 'rb') as f:
        X_train, Y_train, grp_id_train = pickle.load(f)
    with open(os.path.join(savedir, 'test_data'), 'rb') as f:
        X_test, Y_test, grp_id_test = pickle.load(f)
    return X_train, Y_train, grp_id_train, X_test, Y_test, grp_id_test

In [3]:

X_train, Y_train, grp_id_train, X_test, Y_test, grp_id_test = load_data()

train_data = NormData.from_ndarrays(name="train",
                                    X=X_train,
                                    y=Y_train,
                                    batch_effects=grp_id_train)

In [4]:
# Create a NormConf object
norm_conf = NormConf(savemodel=True, 
                    save_dir="resources/save_dir", 
                    log_dir="resources/log_dir",
                    inscaler="standardize",
                    outscaler="standardize",
                    basis_function="bspline",
                    order=3,
                    nknots=5)

Configuration of normative model is valid.


## Configure the regression model

HBR models need to specificy (possibly recursive) parameter configurations. Here, we configure a HBR model with a SHASHb likelihood, a bspline regression in `mu` and `sigma`, and a random effect in the intercept of `mu`. Note that because sigma has to be strictly positive, we specify a `softplus` mapping, so that the output of the linear regression is mapped to the positive domain. 

In [5]:

model_confs = {"M1":{"random_intercept_mu":False,"likelihood":"Normal"},
               "M2":{"random_intercept_mu":True,"likelihood":"Normal"},
               "M3":{"random_intercept_mu":True,"likelihood":"SHASHo"},
               "M4":{"random_intercept_mu":True,"likelihood":"SHASHb"}}

# model_confs = {               "M3":{"random_intercept_mu":True,"likelihood":"SHASHo"}, }


# hbrdata = NormHBR.normdata_to_hbrdata(train_data)

for model_name, model_conf in model_confs.items():
    print("Model: " + model_name)
    print("Constructing model")

    hbr_conf = HBRConf.from_args({"draws": 1500, "tune": 500, "chains": 4, "cores": 16, 
                                  "linear_mu":True,
                                  "linear_sigma":False,
                                  "linear_epsilon":False,
                                  "linear_delta":False,
                                  "random_slope_mu":False,
                                  "random_intercept_sigma":False,
                                  "random_slope_sigma":False,
                                  "random_intercept_epsilon":False,
                                  "random_slope_epsilon":False,
                                  "random_intercept_delta":False,
                                  "random_slope_delta":False,
                                  "random_delta":False,
                                  "random_epsilon":False,
                                  "random_sigma":False} | model_confs[model_name])
    norm_model = NormHBR(norm_conf, hbr_conf)
    norm_model.prepare("response_var_0")
    norm_model.preprocess(train_data)
    hbrdata = norm_model.normdata_to_hbrdata(train_data)
    norm_model.current_regression_model.create_pymc_graph(hbrdata)
    norm_model.current_regression_model.pymc_model.to_graphviz(save=os.path.join(savedir, "refactor", model_name, "model.png"))

    pymc_model = norm_model.current_regression_model.pymc_model
    os.makedirs(os.path.join(savedir, "refactor", model_name), exist_ok=True)

    print("Profiling logp")
    logp_profile = pymc_model.profile(pymc_model.logp())
    print("Saving logp profile pickle")
    with open(os.path.join(savedir, "refactor", model_name, "logp_profile.pkl"), 'wb') as f:
        pickle.dump(logp_profile, f)
    print("Saving logp profile txt")
    with open(os.path.join(savedir, "refactor", model_name, "logp_profile.txt"), 'w') as f:
        logp_profile.summary(f)

    print("Profiling grad")
    grad_profile = pymc_model.profile(pm.gradient(pymc_model.logp()))
    print("Saving grad profile pickle")
    with open(os.path.join(savedir, "refactor", model_name, "grad_profile.pkl"), 'wb') as f:
        pickle.dump(grad_profile, f)
    print("Saving grad profile txt")
    with open(os.path.join(savedir, "refactor", model_name, "grad_profile.txt"), 'w') as f:
        grad_profile.summary(f)

    print("Saving visualization of logp")
    pydotprint(pymc_model.logp(), os.path.join(savedir, "refactor", model_name, "logp.png"))

    print("Saving visualization of grad")
    pydotprint(pm.gradient(pymc_model.logp()), os.path.join(savedir, "refactor", model_name, "logp_grad.png"))





Model: M1
Constructing model
Configuration of regression model is valid.
Profiling logp
Saving logp profile pickle
Saving logp profile txt
Profiling grad
Saving grad profile pickle
Saving grad profile txt
Saving visualization of logp
The output file is available at /home/guus/Desktop/pcn_profile_data/refactor/M1/logp.png
Saving visualization of grad
The output file is available at /home/guus/Desktop/pcn_profile_data/refactor/M1/logp_grad.png
Model: M2
Constructing model
Configuration of regression model is valid.
Profiling logp
Saving logp profile pickle
Saving logp profile txt
Profiling grad
Saving grad profile pickle
Saving grad profile txt
Saving visualization of logp
The output file is available at /home/guus/Desktop/pcn_profile_data/refactor/M2/logp.png
Saving visualization of grad
The output file is available at /home/guus/Desktop/pcn_profile_data/refactor/M2/logp_grad.png
Model: M3
Constructing model
Configuration of regression model is valid.
Profiling logp
Saving logp profile 

In [6]:
def get_string_max_of(thing):
    the_max = max(thing.items(), key=lambda x: x[1])
    return str(the_max[0]), the_max[1]

In [7]:
def extract_data(data:ProfileStats):
    data_dict = {}
    data_dict['compile_time'] = data.compile_time
    data_dict['max_op_time_name'],data_dict['max_op_time'] = get_string_max_of(data.op_time())
    data_dict['max_class_time_name'],data_dict['max_class_time'] =   get_string_max_of(data.class_time())
    data_dict['max_apply_time_name'],data_dict['max_apply_time'] =  get_string_max_of(data.apply_time)
    data_dict['max_compute_total_times_name'],data_dict['max_compute_total_times'] = get_string_max_of(data.compute_total_times())
    data_dict['nb_nodes'] = data.nb_nodes
    data_dict['fct_call_time']  = data.fct_call_time
    return data_dict

In [8]:
# Load the profiles
model_confs = {"M1":{"random_intercept_mu":'False',"likelihood":"Normal"},
               "M2":{"random_intercept_mu":'True',"likelihood":"Normal"},
               "M3":{"random_intercept_mu":'True',"likelihood":"SHASHo"},
               "M4":{"random_intercept_mu":'True',"likelihood":"SHASHb"}}

data_dict = {}

for model_name, model_conf in model_confs.items():

    with open(os.path.join(savedir, 'refactor', model_name, 'logp_profile.pkl'), 'rb') as a:
        logp_profile = pickle.load(a)

    with open(os.path.join(savedir, 'refactor', model_name, 'grad_profile.pkl'), 'rb') as a:
        grad_profile = pickle.load(a)

    print(f"Model: {model_name}")
    data_dict[('refactor',model_name, 'logp_profile')] = extract_data(logp_profile)
    data_dict[('refactor',model_name, 'grad_profile')] = extract_data(grad_profile)
    

Model: M1
Model: M2
Model: M3
Model: M4


In [9]:
with open(os.path.join(savedir, 'refactor', 'data_dict.pkl'), 'wb') as f:
    pickle.dump(data_dict, f)